# Installation

You can install SimplestMaps with pip:

`pip3 install simplestmaps`

(requires python3)

# Usage

First you need to import some basic utilities that SimplestMaps provides:

In [1]:
from simplestmaps import (
    draw_map, 
    marker,
    dot,
    label,
    html,
    line,
    area,
    geojson,
)

And then you can use those utilities to plot all kinds of objects:

# Drawing points

**Markers** are the most basic way of showing a point (coordinates are always lat lon):

In [2]:
draw_map(
    marker(-20, 30),
    marker(40, -100, popup="hello world!"),
)

**Dots** are a bit more customizable and less intrussive by default (sizes always in pixels):

In [3]:
draw_map(
    dot(-20, 30),  # by default: small blue dot
    dot(40, -100, popup="hello world!", color="green", radius=10, opacity=0.5, border_color="yellow", border_width=2),
)

You can also draw **text labels** or even **custom html** in the locations of your points:

In [4]:
draw_map(
    label(-20, 30, text="hello world!"),  # by default: small blue text
    label(30, -75, text="Words", font="Verdana", size=30, color="green", opacity=0.9, popup="this is customizable!"),

    html(10, -40, code='<ul style="color:red"><li><strong>apples</strong></li><li>oranges</li></ul>', popup="custom html markers!"),
)

# Lines and Areas

Lines and areas are super easy, they're just sequences of point coordinates and also quite customizable:

In [5]:
draw_map(
    line([(-35, -65), (5, -70), (40, -100)]),  # by default: thin line
    line([(20, 0), (20, 30), (-20, 30)], color="green", width=10, opacity=0.5, popup="hi!"),

    area([(25, -80), (32, -64), (18, -66)]),  # by default: blue area
    area([(30, 0), (30, 25), (50, 15), (50, 0)], color="green", opacity=0.2, border_color="red", border_width=2, popup="hi!"),
)

# Geojson files!

It's super simple to just plot the contents of geojson files. 

To customize the elements from the geojson, you can use the same functions used to create normal elements:

- points can be configured to be shown as markers, dots, labels, or htmls
- lines can be customized with the lines function
- and areas can be customized with the area function

In [6]:
draw_map(
    geojson("./readme_example.geojson"),

    geojson("./demo.geojson", 
            points_as=label(text="X", color="green"),
            lines_as=line(color="cyan", width=20, opacity=0.3),
            areas_as=area(color="yellow", border_color="black")),
)

# Customizing the map view

SimplestMaps uses Folium under the hood, so you can specify a few parameters that are directly translated to the Folium map parameters to customize how it's presented. Right now, you can set the coordinates of the map center, its zoom level, and the tiles to display.

(more supported tiles: https://python-visualization.github.io/folium/latest/user_guide/raster_layers/tiles.html )

In [7]:
draw_map(
    geojson("./readme_example.geojson"),
    center=(-31.27, -61.49), zoom=14, tiles="OpenStreetMap",
)

# Sets of points, areas or lines

If you have a set of points, lines, areas or geojsons for which you want to use the same style without having to call the helper functions for each element (like in a list comprehension), you can instead use these other helpers. They work just like the single-element versions, except they expect a list of elements as the first argument:

In [8]:
from simplestmaps import (
    marker_set,
    dot_set,
    label_set,
    html_set,
    line_set,
    area_set,
    geojson_set,
)

draw_map(
    dot_set([(0, -80), (0, 0), (0, 80)], color="green", radius=10),
    line_set([
        [(10, -100), (10, 100)],
        [(-10, -100), (-10, 100)],
    ], color="yellow"),
)

# Integration with Shapely

📣 New in v1.3.0! 📣

Shapely is supported out of the box too! You can use its geometries instead of coordinates tuples.

Remember to use the "_set" helpers (dot_set, area_set, etc) for shapely "Multi" things (MultiPoint, MultiPolygon, etc).

SimplestMaps also natively supports Telluric, Satellogic's own lib for geo data, in a very similar way (just remember to use "_set" helpers when dealing with GeoVectors that contain shapely's "Multi" things).

In [9]:
from shapely import Point, LineString, LinearRing, Polygon, MultiPoint, MultiPolygon, MultiLineString

# all shapely coordinates are lon, lat!
shp_point = Point(-65, -35)
shp_multi_point = MultiPoint([[0, 40], [10, 40], [20, 40]])

shp_line = LineString([[0, 10], [15, 10], [15, 5]])
shp_linear_ring = LinearRing([[0, 0], [-5, -5], [-5, 10]])
shp_multi_line = MultiLineString([
    [[50, 30], [60, 40]],
    [[55, 30], [65, 40]],
    [[60, 30], [70, 40]],
])
shp_multi_linear_ring = MultiLineString([
    [[75, 30], [85, 40], [75, 50]],
    [[80, 30], [90, 40], [80, 50]],
    [[85, 30], [95, 40], [85, 50]],
])

shp_polygon = Polygon([[-100, 30], [-100, 40], [-80, 35]])
shp_multi_polygon = MultiPolygon([
    Polygon([(-50, 30), (-50, 40), (-70, 30)]),
    Polygon([(-55, 35), (-55, 45), (-75, 35)]),
])

draw_map(
    dot(shp_point, color="red", popup="a point"),
    dot_set(shp_multi_point, color="black", popup="a multi point"),
    
    line(shp_line, color="green", popup="a line"),
    line(shp_linear_ring, color="orange", popup="a linear ring"),
    line_set(shp_multi_line, color="purple", popup="a multi line"),
    line_set(shp_multi_linear_ring, color="magenta", popup="a multi linear ring"),

    area(shp_polygon, color="cyan", popup="a polygon"),
    area_set(shp_multi_polygon, color="teal", popup="a multi polygon"),
)

# Custom geometry objects instead of (lat,lon) tuples

📣 New in v1.2.0! 📣

SimplestMaps supports not only (lat,lon) tuples and shapely objects, but any kind of custom coordinate and geometry containing objects you might have. Some are supported out of the box, and others require just a tiny bit of work to make them compatible.

If you have a class that represents coordinates and has either:

- "lat" and "lon" attributes
- "latitude" and "longitude" attributes
- or "latitude_deg" and "longitude_deg" attributes

Then SimplestMaps already supports them out of the box in all the places where you can use tuples.

For instance:

In [10]:
class House:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude

house_a = House(-35, -65)
house_b = House(5, -70)
house_c = House(40, -100)

draw_map(
    marker(house_a),
    marker(house_b),
    marker(house_c),
    line([house_a, house_b, house_c]),
)

If your custom geometry objects don't follow those attribute naming conventions or represent more complex types like areas or lines, you can simply extract (lat,lon) tuples and use them as inputs for SimplestMaps. But that's tedious if you are doing lots of maps with your custom types.

If that the case, you can tell SimplestMaps how to convert them to avoid having to manually do it each time you need to draw a map. You only need to write a function that is able to receive an instance of your custom type, and return either a single tuple of (lat,lon) coordinates, or a sequence of tuples of (lat,lon) coordinates. 

Like this:

(notice how the converter function used for QuarantineZone can just return a list of House instances, because we also registered a converter for House. SimplestMaps tries to save you work as much as it can :) )

In [11]:
from simplestmaps import auto_convert

class House:
    def __init__(self, entrance_latitude, entrance_longitude):
        self.entrance_latitude = entrance_latitude
        self.entrance_longitude = entrance_longitude
        
class QuarantineZone:
    def __init__(self, houses):
        self.houses = houses


auto_convert(House, lambda h: (h.entrance_latitude, h.entrance_longitude))
auto_convert(QuarantineZone, lambda q: q.houses)

house_a = House(-35, -65)
house_b = House(5, -70)
house_c = House(40, -100)
quarantine_zone = QuarantineZone([house_a, house_b, house_c])

draw_map(
    marker(house_a),
    marker(house_b),
    marker(house_c),
    area(quarantine_zone, color="red"),
)